# Numerics of Machine Learning
# Exercise Sheet No. 12 — Second-Order Optimization

---
University of Tübingen, Winter Term 2022/23
&copy; N. Bosch, J. Grosse, P. Hennig, A. Kristiadi, M. Pförtner, J. Schmidt, F. Schneider, L. Tatzel, J. Wenger, 2022 CC BY-NC-SA 3.0

---

PLEASE FILL OUT FOR ALL TEAM MEMBERS:
- Last name, first name, Matrikelnr.
- Last name, first name, Matrikelnr.

The homework must be submitted as a `.pdf` on ILIAS. We do not grade `.ipynb` submissions.

**Overview:** 
This week's coding exercise is concerned with the Hessian-free method and the role of damping in stochastic second-order optimization. There are three parts:
1. First, we will implement a basic version of the Hessian-free optimizer in PyTorch. If you want to know more about this optimizer, you can find the paper [here](https://www.cs.toronto.edu/~jmartens/docs/Deep_HessianFree.pdf). 
2. Next, we will apply this optimizer to a small neural network. 
3. Finally, we will investigate the relation between damping and its dependence on the mini-batch size. 

In [ ]:
import copy
import math
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# Make deterministic
random_seed = 0
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True

In [ ]:
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"  # matplotlib bugfix

## 1. Hessian-free optimizer in PyTorch

Below, we will implement a second-order method that was discussed in the lecture: the Hessian-free (HF) optimizer. This optimizer uses the method of conjugate gradients to approximate the Newton step. Note that a lot of functionality (e.g. the `cg`-method, the GGN-vector product, and functions for converting the model's parameters to a vector and vice versa) is already implemented in `hf_utils.py`. 

**Your task:** Complete the missing bits of code below (marked with `TODO`).

In [ ]:
from torch.nn.utils.convert_parameters import parameters_to_vector

from hf_utils import (
    cg,
    ggn_vector_product,
    vector_to_parameter_list,
    vector_to_trainparams,
)


class HessianFree(torch.optim.Optimizer):
    """This class implements a basic version of a Hessian-free optimizer in 
    Pytorch.
    """

    def __init__(
        self,
        params,
        damping=0.0,
        cg_maxiter=20,
        lr=1.0,
        verbose=False,
    ):
        """The constructor creates an instance of the `HessianFree` optimizer.
        
        Args:
            params: An iterable of `torch.Tensor`s that represents the 
                network's parameters.
            damping: A scalar multiple of the identity matrix is added to the
                curvature matrix when the cg-method is applied. This damping 
                parameter is kept constant.
            cg_max_iter: The maximum number of cg-iterations. 
            lr: The learning rate. The update step is scaled by this scalar. 
            verbose (bool): Print information during the computations.
        """

        # Check hyperparameters
        if damping < 0.0:
            raise ValueError(f"Invalid damping = {damping}")
        if cg_maxiter < 1:
            raise ValueError(f"Invalid cg_max_iter: {cg_max_iter}")
        if lr < 0.0 or lr > 1.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        
        # Call parent class constructor
        defaults = dict(damping=damping, cg_maxiter=cg_maxiter, lr=lr)
        super().__init__(params, defaults)

        # Only one parameter group is supported
        if len(self.param_groups) != 1:
            error_msg = "`HessianFree` does not support per-parameter options."
            raise ValueError(error_msg)

        self.verbose = verbose
        self._group = self.param_groups[0]
        self._params = self._group["params"]

        # All computations are performed in the subspace of trainable parameters
        self._params_list = [p for p in self._params if p.requires_grad]
        self.device = self._params_list[0].device
        
        # Print info 
        if self.verbose:
            num_params = sum(p.numel() for p in self._params_list)
            print("Number of trainable parameters: ", num_params)
            print("Device = ", self.device)
    
    
    @staticmethod
    def _Gv(loss, outputs, params_list, vec):
        """Return the GGN-vector product with the vector `vec`."""
        
        ### TODO: Implement the GGN-vector product. 
    
    
    def step(self, forward):
        """Perform an update step.
        
        Args:
            forward: This function returns a `(loss, outputs)`-tuple,
                where `loss` is the target function value. Here is a pseudo-code
                example of the training loop:
                ```
                for step_idx in range(num_steps):
                    inputs, targets = get_minibatch_data()
                    
                    def forward():
                        outputs = model(inputs)
                        loss = loss_function(outputs, targets)
                        return loss, outputs
                    
                    opt.step(forward=forward)
                ```
        """
        
        if self.verbose:
            print(f"\nComputing HF update step...")
        
        
        # ----------------------------------------------------------------------
        # Set up the linear system
        # ----------------------------------------------------------------------

        # Evaluate the loss
        loss, outputs = forward()
        init_loss = loss.item()

        # Evaluate the gradient
        grad = torch.autograd.grad(
            loss, self._params_list, create_graph=True, retain_graph=True
        )
        grad = parameters_to_vector(grad).detach()
        
        # Matrix-vector products with the curvature matrix
        def mvp(x):
            return self._Gv(loss, outputs, self._params_list, x)
        
        
        # ----------------------------------------------------------------------
        # Apply cg
        # ----------------------------------------------------------------------
        damping = self._group["damping"]
        cg_maxiter = self._group["cg_maxiter"]

        if self.verbose:
            print(f"  Running cg...")
        
        ### TODO: Apply cg (with at most `cg_maxiter` iterations) to the damped (!)
        ### linear system.

        
        # ----------------------------------------------------------------------
        # Parameter update
        # ----------------------------------------------------------------------
        lr = self._group["lr"]
        
        ### TODO: Scale the Newton step with the learning rate, update the parameters
        
        # Print initial and final loss
        final_loss = forward()[0].item()
        if self.verbose:
            msg = f"  Initial loss = {init_loss:.6f} --> "
            msg += f"final loss = {final_loss:.6f}"
            print(msg)
            
        return 

## 2. Apply HF to a simple test problem

Before we can apply our optimizer, we have to define a test problem. We will use a simple neural network on MNIST data.

### 2.1 Preparations

In [ ]:
# Download data set to this folder
data_dir = os.path.join(os.getcwd(), "data")

# Normalize data
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        (0.1307,), (0.3081,))
])

# Training and test data
train_data = MNIST(data_dir, train=True, download=True, transform=transform)
test_data = MNIST(data_dir, train=False, download=True, transform=transform)

# Data loaders for training and test data
def get_train_loader(batch_size):
    return torch.utils.data.DataLoader(
        train_data, 
        batch_size=batch_size, 
        shuffle=True, 
        drop_last=True
    )

def get_test_loader(batch_size):
    return torch.utils.data.DataLoader(
        test_data, 
        batch_size=batch_size, 
        shuffle=False
    )

In [ ]:
# Set up model
def get_model():
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(28 * 28, 10),
        nn.ReLU(),
        nn.Linear(10, 10),
    )

In [ ]:
# Loss-function
loss_func = torch.nn.CrossEntropyLoss(reduction="mean")
loss_func_no_reduction = torch.nn.CrossEntropyLoss(reduction="none")

In [ ]:
def eval_mb_metrics(model, inputs, targets):
    """Compute the loss and accuracy on the mini-batch defined by `inputs` 
    and `targets`.
    """
    
    # Put model into evaluation mode
    model.eval()
    
    with torch.no_grad():
        
        # Forward pass, compute loss and accuracy
        outputs = model(inputs)
        loss = loss_func(outputs, targets)
        predictions = torch.argmax(outputs, 1)
        acc = (predictions == targets).sum() / len(targets)
        return loss.item(), acc.item()
    

def eval_test_metrics(model, batchsize_test):
    """Compute the loss and accuracy on the test set."""
    
    # Put model into evaluation mode
    model.eval()
    
    loss = 0.0
    correct_counter = 0
    data_counter = 0
    
    with torch.no_grad():
        for inputs, targets in get_test_loader(batchsize_test):
            
            # Forward pass, aggregate loss and correct predictions
            outputs = model(inputs)
            loss += loss_func_no_reduction(outputs, targets).sum().item()
            predictions = torch.argmax(outputs, 1) 
            correct_counter += (predictions == targets).sum().item()
            data_counter += len(targets)
    
    loss /= data_counter
    acc = correct_counter / data_counter
    
    return loss, acc

### 2.2. Run HF

Let's apply the optimizer to our test problem. We have to perform multiple training runs with the HF-optimizer in the next part of the tutorial, so it makes sense to implement a reusable `run_hf`-function. This function runs the Hessian-free optimizer for a given number of steps `num_steps`. It contains a training loop and stores and returns the training metrics (mini-batch loss/accuracy and test loss/accuracy).

**Your task:** Complete the missing bits of code below (marked with `TODO`). 

In [ ]:
def log_mb_metrics(model, inputs, targets, step, results_dict):
    """Compute the mini-batch loss and accuracy in step `step` and 
    put the results into `results_dict`.
    """
    loss, acc = eval_mb_metrics(model, inputs, targets)
    
    # Store results
    results_dict["step"].append(step)
    results_dict["loss"].append(loss)
    results_dict["acc"].append(acc)

    
def log_test_metrics(model, batchsize_test, step, results_dict):
    """Compute the test loss and accuracy in step `step` and 
    put the results into `results_dict`.
    """
    loss, acc = eval_test_metrics(model, batchsize_test)
    
    # Store results
    results_dict["step"].append(step)
    results_dict["loss"].append(loss)
    results_dict["acc"].append(acc)


def run_hf(
    model,
    opt,
    batchsize_train,
    batchsize_test,
    num_steps,
    log_test_metrics_every
):
    """Run the Hessian-free optimizer `opt` for `num_steps` steps. The test
    loss and accuracy are evaluated every `log_test_metrics_every` steps.
    """
    
    # Track these quantities
    mb_metrics_before = {"step": [], "loss": [], "acc": []}
    mb_metrics_after = {"step": [], "loss": [], "acc": []}
    test_metrics_after = {"step": [], "loss": [], "acc": []}
        
    # Training loop
    train_loader = get_train_loader(batchsize_train)
    step = 0
    while True:
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            
            # Log before
            log_mb_metrics(model, inputs, targets, step, mb_metrics_before)
                
            # Put model into training mode
            model.train()
            
            ### TODO: Compute and apply the HF step
            
            # Log after
            log_mb_metrics(model, inputs, targets, step, mb_metrics_after)
            if step % log_test_metrics_every == 0:
                log_test_metrics(model, batchsize_test, step, test_metrics_after)
            
            # Increment step counter
            step += 1
            
            # Break from training loop after `num_steps` steps
            if step >= num_steps:
                break
        if step >= num_steps:
            break
    
    # Return training metrics
    return mb_metrics_before, mb_metrics_after, test_metrics_after

In [ ]:
# Get model, optimizer, data
model = get_model()
opt = HessianFree(
    model.parameters(),
    damping=1.0,
    cg_maxiter=20,
    lr=1.0,
    verbose=False
)

# Apply the optimizer to the test problem
metrics = run_hf(
    model, 
    opt, 
    batchsize_train=100, 
    batchsize_test=1000,
    num_steps=20, 
    log_test_metrics_every=1
)
mb_metrics_before, mb_metrics_after, test_metrics_after = metrics

**Your task:** Visualize the three metrics returned by `run_hf` in two subplots (one containing the losses, the other one the accuracies). Is the per-step progress on the mini-batch always reflected in similar progress on the test set? If not, do you have an explanation for this observation?

In [ ]:
### TODO: Visualization

**Your answer:** TODO

## 3. Batch size and damping

The Hessian-free optimizer uses a stochastic estimate of the gradient and the curvature. The noise on these estimates depends on the size of the mini batch: The more data is used, the more accurate the estimates. If our estimates are very noisy, we might want to restrict the optimizer to smaller updates, i.e. use larger damping. So, the optimal damping seems to depend on the mini-batch size. Let's investigate this dependency in more detail. 

First, we define a set of five different batch sizes (`50`, `100`, `200`, `400` and `1000`) and a set of four damping values (`5e-2`, `1e-1`, `5e-1`, and `1.0`). For each combination of batch size and damping, we run the HF-optimizer and evaluate the *best* test loss and test accuracy that was achieved during the HF-run. For a fair comparison between multiple runs with different batch sizes, we choose the number of steps such that `batch_size * num_steps` is constant for all runs (see `batch_size_to_steps` below).

**Your task:** Complete the missing bits of code below (marked with `TODO`).

In [ ]:
# Get a fresh model
model_orig = get_model()

# Key = batch size, 
# Value = number of steps
batch_size_to_steps = {
    50: 40,
    100: 20,
    200: 10,
    400: 5,
    1000: 2,
}
dampings = [5e-2, 1e-1, 5e-1, 1.0]

# Best test loss/acc for all batch size damping combinations
best_test_loss = torch.empty(len(batch_size_to_steps), len(dampings))
best_test_acc = torch.empty(len(batch_size_to_steps), len(dampings))

# Go through all combinations of batch size and damping
for bs_idx, (batch_size, num_steps) in enumerate(batch_size_to_steps.items()):    
    for damping_idx, damping in enumerate(dampings):
        
        # Set up problem (copy model!!)
        model = copy.deepcopy(model_orig)
        opt = HessianFree(
            model.parameters(),
            damping=damping,
            cg_maxiter=20,
            lr=1.0,
            verbose=False
        )
        
        # Print info
        log_every=int(math.ceil(num_steps/10))
        info_msg = f"batch_size = {batch_size}, num_steps = {num_steps}, "
        info_msg += f"damping = {damping}, log_every = {log_every}"
        print(info_msg)
        
        ### TODO: Run HF, store the best test loss/acc achieved within the given 
        ### number of steps in `best_test_loss`/`best_test_acc`

**Your task:** Visualize the optimal damping (in terms of test loss and test accuracy) for each batch size. 

In [ ]:
### TODO: Visualization

**Your task:** For each batch size, plot the best test loss and test accuracy (in two separate plots) achieved using the ideal damping. Write down your observations and possible explanations. 

In [ ]:
### TODO: Visualization

**Your answer:** TODO